In [1]:
import os 

WORKING_DIR = os.getcwd()

print("Working Directory:", WORKING_DIR)


Working Directory: /kaggle/working


In [2]:
!python --version

Python 3.10.12


# 0. Install and Import Dependencies

In [ ]:
# !pip list

In [ ]:
# !pip install opencv-python==4.6.0.66 matplotlib==3.6.2 imageio==2.23.0 gdown==4.6.0 tensorflow==2.12 

In [ ]:
!pip list

In [4]:
!pip install tensorflow==2.16.1 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorfl

In [ ]:
!nvidia-smi

In [ ]:
!lsb_release -a

In [2]:
# import os 

# WORKING_DIR = os.getcwd()

# print("Working Directory:", WORKING_DIR)


import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
from tqdm.notebook import tqdm

In [3]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [4]:
print("TensorFlow version:", tf.__version__)
print("GPU detected:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.17.1
GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [ ]:
import os 

os.remove('/kaggle/working/data.zip')

# 1. Build Data Loading Functions

In [26]:
import gdown

In [24]:
import shutil 

shutil.rmtree('/kaggle/working/data')
os.remove('/kaggle/working/data.zip')

In [28]:
# url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
# output = 'data.zip'
# gdown.download(url, output, quiet=False)
# gdown.extractall('data.zip')

In [29]:
corrupted = []

In [30]:
import subprocess

# check if the video is corrupted or not
def is_video_corrupted(path: str) -> bool:
    # FFmpeg command to detect video errors
    command = ["ffmpeg", "-v", "error", "-i", path, "-f", "null", "-"]
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.stderr:
            print(f"Errors found in video {path}:\n{result.stderr}")
            return True
    except Exception as e:
        print(f"Error running FFmpeg: {e}")
        return True
    return False

def get_corrupted():
    for file in os.listdir(os.path.join(WORKING_DIR,'data', 's1')):
        #print(file)
        path = os.path.join(WORKING_DIR,'data', 's1',f'{file}')
        # print(f"Processing file: {file}")
        if is_video_corrupted(path):
            corrupted.append(file)
            
get_corrupted()

Errors found in video /kaggle/working/data/s1/Thumbs.db:
/kaggle/working/data/s1/Thumbs.db: Invalid data found when processing input

Errors found in video /kaggle/working/data/s1/sgib8n.mpg:
[mpeg1video @ 0x57a3fa44bf40] ac-tex damaged at 22 17
[mpeg1video @ 0x57a3fa44bf40] Warning MVs not available



In [31]:
corrupted

['Thumbs.db', 'sgib8n.mpg']

In [34]:
for file in corrupted:
    if(file.endswith('.mpg')):
        os.remove(os.path.join(WORKING_DIR,'data', 's1',f'{file}'))
        os.remove(os.path.join(WORKING_DIR,'data', 'alignments','s1',f"{file.split('.')[0]}.align"))
    else:
        os.remove(os.path.join(WORKING_DIR,'data', 's1',f'{file}'))

In [6]:
def count_what(entity:str):
    count = 0
    if entity == 'videos':
        for video in os.listdir(os.path.join('data/s1')):
            count += 1
    elif entity == 'allignments':
        for allign in os.listdir(os.path.join('data/alignments/s1')):
            count += 1
    else:
        print("Select between videos and allignments!")
        return
            
    return count

count_what('videos')

999

In [4]:
def load_video(path:str) -> List[float]: 

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))): 
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Warning: Skipping empty or unreadable frame.")
            continue
        # frame = tf.image.rgb_to_grayscale(frame)
        # frames.append(frame[190:236,80:220,:])
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        roi = frame[190:236,80:220]
        print(roi.shape)
        # frame = cv2.resize(roi, (140, 46))
        frame_i = tf.expand_dims(roi, axis = -1)
        print(frame_i.shape)
        frames.append(frame_i)
    cap.release()

    frames = np.array(frames)

    # Normalize frames
    mean = np.mean(frames)
    std = np.std(frames) + 1e-8
    normalized_frames = (frames - mean) / (std)  # Prevent divide by zero
    return normalized_frames
    
    # mean = tf.math.reduce_mean(frames)
    # std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    # return tf.cast((frames - mean), tf.float32) / std

In [ ]:
def load_video(path:str) -> List[float]: 

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))): 
        ret, frame = cap.read()
        # frame = tf.image.rgb_to_grayscale(frame)
        # frames.append(frame[190:236,80:220,:])
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        roi = frame[190:236,80:220]
        print(roi.shape)
        # frame = cv2.resize(roi, (140, 46))
        frame_i = tf.expand_dims(roi, axis = -1)
        print(frame_i.shape)
        frames.append(frame_i)
    cap.release()

    frames = np.array(frames)

    # Normalize frames
    mean = np.mean(frames)
    std = np.std(frames) + 1e-8
    normalized_frames = (frames - mean) / (std)  # Prevent divide by zero
    return normalized_frames
    
    # mean = tf.math.reduce_mean(frames)
    # std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    # return tf.cast((frames - mean), tf.float32) / std

vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': 
            tokens = [*tokens,' ',line[2]]
    # temp = tf.strings.unicode_split(tokens, input_encoding='UTF-8')
    # print(temp)
    # print(temp.shape)
    # temp = tf.reshape(tf.strings.unicode_split(temp, input_encoding='UTF-8'), (-1))
    # print(temp)
    # print(temp.shape)
    result = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]
    # print(result.shape)
    result = tf.expand_dims(result, axis = 0)
    return result

In [22]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '] (size =40)


In [ ]:
# char_to_num.get_vocabulary()

In [ ]:
char_to_num(['n','i','c','k'])

In [ ]:
num_to_char([14,  9,  3, 11])

In [7]:
lengths = []

In [8]:
for file in os.listdir('/kaggle/working/data/alignments/s1'):
    path = os.path.join('/kaggle/working/data/alignments/s1',file)
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': 
            tokens = [*tokens,' ',line[2]]

    result = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))
    lengths.append(result.shape)

In [12]:
path = os.path.join('/kaggle/working/data/alignments/s1',file)
with open(path, 'r') as f: 
    lines = f.readlines() 
tokens = []
for line in lines:
    line = line.split()
    if line[2] != 'sil': 
        tokens = [*tokens,' ',line[2]]

result = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]
print(result)

tf.Tensor([16 12  1  3  5 39 18  5  4 39  9 14 39 22 39  6 15 21 18 39 14 15 23], shape=(23,), dtype=int64)


In [9]:
for i in lengths:
    if i[0] < 20:
        print(i)

In [23]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': 
            tokens = [*tokens,' ',line[2]]
    # temp = tf.strings.unicode_split(tokens, input_encoding='UTF-8')
    # print(temp)
    # print(temp.shape)
    # temp = tf.reshape(tf.strings.unicode_split(temp, input_encoding='UTF-8'), (-1))
    # print(temp)
    # print(temp.shape)
    result = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]
    # print(result.shape)
    result = tf.expand_dims(result, axis = 0)
    return result

In [ ]:
os.listdir('/kaggle/working/data/alignments/s1')[:10]

In [5]:
def get_word_corpus(directory:str):
    corpus_freq = {}
    for file in os.listdir(directory):
        file_path = os.path.join(directory,file)
        with open(file_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            line = line.split()
            word = line[2]
            if word != 'sil':
                corpus_freq[word] = corpus_freq.get(word, 0) + 1

    return corpus_freq

corpus_freq1 = get_word_corpus('/kaggle/working/data/alignments/s1')
print(corpus_freq1)

{'set': 256, 'white': 250, 'by': 248, 'c': 40, 'two': 100, 'please': 250, 'red': 250, 'at': 252, 'o': 40, 'one': 100, 'again': 250, 'bin': 240, 'blue': 250, 'in': 252, 'z': 40, 'zero': 100, 'now': 250, 'place': 256, 'x': 40, 'soon': 250, 'with': 248, 'i': 40, 'four': 100, 'green': 250, 'n': 40, 'eight': 100, 't': 40, 'nine': 100, 'q': 40, 'h': 40, 'lay': 248, 'sp': 24, 'e': 40, 'six': 100, 's': 40, 'five': 100, 'j': 40, 'd': 40, 'seven': 100, 'y': 40, 'a': 40, 'three': 100, 'f': 40, 'l': 40, 'u': 40, 'm': 40, 'k': 40, 'r': 40, 'g': 40, 'v': 40, 'b': 40, 'p': 40}


In [6]:
corpus_freq2 = get_word_corpus('/kaggle/working/set1/Alignments')
print(corpus_freq2)

{'set': 1013, 'green': 999, 'by': 1002, 'v': 160, 'nine': 400, 'again': 999, 'place': 1024, 'blue': 1000, 'with': 992, 'x': 160, 'two': 400, 'bin': 960, 'u': 160, 'now': 1000, 'red': 999, 'at': 1007, 'i': 160, 'please': 1000, 'lay': 1001, 'e': 160, 'three': 400, 'white': 1000, 'r': 160, 'm': 160, 'seven': 400, 'c': 160, 'd': 160, 'in': 997, 'z': 160, 'sp': 252, 't': 160, 'one': 400, 'soon': 999, 'n': 159, 'f': 160, 'o': 160, 'b': 160, 'j': 160, 's': 160, 'six': 399, 'a': 160, 'eight': 400, 'zero': 400, 'q': 160, 'k': 160, 'five': 400, 'p': 160, 'y': 160, 'g': 160, 'four': 399, 'h': 160, 'l': 159}


In [4]:
corpus = []
for key in corpus_freq.keys():
    corpus.append(key)

print(corpus)

['lay', 'blue', 'with', 'r', 'four', 'please', 'bin', 'white', 'by', 'a', 'seven', 'again', 'place', 'red', 'in', 'c', 'two', 'now', 'k', 'at', 'one', 'soon', 'six', 'green', 't', 'set', 'v', 'b', 'five', 'g', 'zero', 'd', 'eight', 'l', 'n', 'j', 'three', 's', 'u', 'q', 'nine', 'x', 'f', 'p', 'i', 'y', 'sp', 'e', 'o', 'h', 'z', 'm']


In [7]:
corpus = []
for key in corpus_freq2.keys():
    corpus.append(key)

print(corpus)

['set', 'green', 'by', 'v', 'nine', 'again', 'place', 'blue', 'with', 'x', 'two', 'bin', 'u', 'now', 'red', 'at', 'i', 'please', 'lay', 'e', 'three', 'white', 'r', 'm', 'seven', 'c', 'd', 'in', 'z', 'sp', 't', 'one', 'soon', 'n', 'f', 'o', 'b', 'j', 's', 'six', 'a', 'eight', 'zero', 'q', 'k', 'five', 'p', 'y', 'g', 'four', 'h', 'l']


In [8]:
len(corpus)

52

In [5]:
import pickle

# Save corpus words as pickle
with open("corpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

In [9]:
import matplotlib.pyplot as plt

# Example dictionary (replace with your GRID corpus)

# Extract words and frequencies
words = list(corpus_freq2.keys())
frequencies = list(corpus_freq2.values())

# Sort words by frequency (descending order for better visualization)
sorted_pairs = sorted(zip(frequencies, words), reverse=True)
sorted_frequencies, sorted_words = zip(*sorted_pairs)

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(sorted_words, sorted_frequencies, color='skyblue')

# Customize the plot
plt.title("Word Frequency Distribution in GRID Dataset", fontsize=14)
plt.xlabel("Words", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotate x-labels for readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show or save the plot
plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()

In [ ]:
len(corpus)

In [ ]:
load_alignments('/kaggle/working/data/alignments/s1/bbaf2n.align')

In [8]:
def load_data(path: str): 
    path = bytes.decode(path.numpy())
    file_name = path.split('/')[-1].split('.')[0]
    # File name splitting for windows
    # file_name = path.split('\\')[-1].split('.')[0]
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join('data','alignments','s1',f'{file_name}.align')
    frames = load_video(video_path) 
    alignments = load_alignments(alignment_path)
    
    return frames, alignments

In [ ]:
test_path = './data/s1/bbal6n.mpg'

In [ ]:
tf.convert_to_tensor(test_path).numpy().decode('utf-8').split('/')[-1].split('.')[0]

In [ ]:
frames, alignments = load_data(tf.convert_to_tensor(test_path))

In [ ]:
plt.imshow(frames[40])

In [ ]:
alignments

In [ ]:
tf.strings.reduce_join([bytes.decode(x) for x in num_to_char(alignments.numpy()).numpy()])

In [9]:
def mappable_function(path:str) ->List[str]:
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

# 2. Create Data Pipeline

In [ ]:
from matplotlib import pyplot as plt

In [10]:
import math

total_videos = count_what('videos')

batch_size = 2

train_size = int(total_videos * 0.9)
test_size = total_videos - train_size

train_batch = math.ceil(train_size / batch_size)
test_batch = math.ceil(test_size / batch_size)

print('Train:',train_size)
print('Test:',test_size)

print("Steps per epoch:",train_batch)
print("Validation steps per epoch:", test_batch)

Train: 899
Test: 100
Steps per epoch: 450
Validation steps per epoch: 50


In [12]:
data = tf.data.Dataset.list_files('./data/s1/*.mpg')
data = data.shuffle(500, reshuffle_each_iteration=False)
data = data.map(mappable_function)
data = data.padded_batch(batch_size, padded_shapes=([75, 46, 140, 1],[1,40]))
data = data.prefetch(tf.data.AUTOTUNE)
data = data.repeat()
# Added for split 
train = data.take(train_batch)
test = data.skip(train_batch)

In [ ]:
test

In [ ]:
type(test)

In [ ]:
count = 0
for frames, alignments in test:
    count += 1
    if(count == 1):
        print(frames.shape)
        print(alignments.shape)
        break

print("Done!")

In [16]:
d = data.as_numpy_iterator().next()

In [116]:
print(d[0].shape)

(2, 75, 46, 140, 1)


In [15]:
print(d[1])

[[[12  1 25 39 18  5  4 39  2 25 39 12 39 15 14  5 39 19 15 15 14  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]

 [[19  5 20 39  2 12 21  5 39  2 25 39  2 39 15 14  5 39  1  7  1  9 14
    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]]


In [118]:
frames.shape

(2, 75, 46, 140, 1)

In [ ]:
alignments.shape

In [ ]:
len(frames)

In [ ]:
sample = data.as_numpy_iterator()

In [ ]:
val = sample.next(); 

In [ ]:
# imageio.mimsave('./animation.gif', val[0][0], fps=10)

In [ ]:
# 0:videos, 0: 1st video out of the batch,  0: return the first frame in the video 
plt.imshow(val[0][0][35])

In [ ]:
tf.strings.reduce_join([num_to_char(word) for word in val[1][0]])

# 3. Design the Deep Neural Network

In [13]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [ ]:
data.as_numpy_iterator().next()[0][0].shape

In [ ]:
model = Sequential()
model.add(Conv3D(128, 3, input_shape=(75,46,140,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(75, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))

In [ ]:
model.summary()

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D

time_steps = 75
height, width, channels = 46, 140, 1

# Input layer
input_layer = Input(shape=(time_steps, height, width, channels))

# Convolutional layers
x = Conv3D(128, 3, padding='same')(input_layer)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(256, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(75, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

# Flatten and TimeDistributed layer
x = TimeDistributed(Flatten())(x)

# Attention layer
# attention_output = Attention(use_scale=True, score_mode='dot')([x, x])

# Bidirectional LSTMs
lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal',return_sequences=True))(x)
lstm_output = Dropout(0.5)(lstm_output)

lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal',return_sequences=True))(lstm_output)
lstm_output = Dropout(0.5)(lstm_output)

# Attention layer
attention_output = Attention(use_scale=True, score_mode='dot')([lstm_output, lstm_output])

# Dense output layer
output_layer = Dense(char_to_num.vocabulary_size() + 1, kernel_initializer='he_normal', activation='softmax')(attention_output)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)

# Print the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 75, 46, 140, 1) │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 75, 46, 140,    │          3,584 │ input_layer_1[0][0]    │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 75, 46, 140,    │              0 │ conv3d_3[0][0]         │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_3           │ (None, 75, 23, 70,     │              0 │ activation_3[0][0]     │
│ (MaxPooling3D)            │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 75, 23, 70,     │        884,992 │ max_pooling3d_3[0][0]  │
│                           │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 75, 23, 70,     │              0 │ conv3d_4[0][0]         │
│                           │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_4           │ (None, 75, 11, 35,     │              0 │ activation_4[0][0]     │
│ (MaxPooling3D)            │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 75, 11, 35, 75) │        518,475 │ max_pooling3d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_5 (Activation) │ (None, 75, 11, 35, 75) │              0 │ conv3d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_5           │ (None, 75, 5, 17, 75)  │              0 │ activation_5[0][0]     │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_1        │ (None, 75, 6375)       │              0 │ max_pooling3d_5[0][0]  │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 75, 256)        │      6,660,096 │ time_distributed_1[0]… │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 75, 256)        │              0 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 75, 256)        │        394,240 │ dropout_2[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├──────────────────────

 Total params: 8,471,925 (32.32 MB)

 Trainable params: 8,471,925 (32.32 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D

batch_size  = 1
height, width, channels = 46, 140, 1

# Input layer
input_layer = Input(batch_shape=(batch_size, None, height, width, channels))

# Convolutional layers
x = Conv3D(128, 3, padding='same')(input_layer)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(256, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(75, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

# Flatten and TimeDistributed layer
x = TimeDistributed(Flatten())(x)

# Attention layer
# attention_output = Attention(use_scale=True, score_mode='dot')([x, x])

# Bidirectional LSTMs
lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal', stateful = True, return_sequences=True))(x)
lstm_output = Dropout(0.5)(lstm_output)

lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal', stateful = True, return_sequences=True))(lstm_output)
lstm_output = Dropout(0.5)(lstm_output)

# Attention layer
attention_output = Attention(use_scale=True, score_mode='dot')([lstm_output, lstm_output])

# Dense output layer
output_layer = Dense(char_to_num.vocabulary_size() + 1, kernel_initializer='he_normal', activation='softmax')(attention_output)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)

# Print the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (1, None, 46, 140, 1)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (1, None, 46, 140,     │          3,584 │ input_layer[0][0]      │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (1, None, 46, 140,     │              0 │ conv3d[0][0]           │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (1, None, 23, 70, 128) │              0 │ activation[0][0]       │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (1, None, 23, 70, 256) │        884,992 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (1, None, 23, 70, 256) │              0 │ conv3d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_1           │ (1, None, 11, 35, 256) │              0 │ activation_1[0][0]     │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (1, None, 11, 35, 75)  │        518,475 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (1, None, 11, 35, 75)  │              0 │ conv3d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_2           │ (1, None, 5, 17, 75)   │              0 │ activation_2[0][0]     │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (1, None, 6375)        │              0 │ max_pooling3d_2[0][0]  │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (1, None, 256)         │      6,660,096 │ time_distributed[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (1, None, 256)         │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (1, None, 256)         │        394,240 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (1, None, 256)         │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)

 Total params: 8,471,925 (32.32 MB)

 Trainable params: 8,471,925 (32.32 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
5*17*75

In [ ]:
val[0].shape

In [ ]:
yhat = model.predict(val[0])

In [ ]:
yhat.shape

In [ ]:
tf.strings.reduce_join([num_to_char(x) for x in tf.argmax(yhat[0],axis=1)])

In [ ]:
tf.strings.reduce_join([num_to_char(tf.argmax(x)) for x in yhat[1]])

In [ ]:
model.input_shape

In [ ]:
model.output_shape

# 4. Setup Training Options and Train

In [ ]:
def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * float(tf.math.exp(-0.1))

In [ ]:
from keras.saving import register_keras_serializable

@register_keras_serializable(name = 'CTCLoss')
def CTCLoss(y_true, y_pred):
    # print(y_true.shape)
    # print(y_pred.shape)
    y_true = tf.squeeze(y_true, axis = 1)
    # print(y_true.shape)
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    # y_true = tf.squeeze(y_true, axis = 1)
    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    # print(input_length.shape)
    # print(label_length.shape)

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
Y_true = alignments
Y_pred = yhat
loss = CTCLoss(Y_true, Y_pred)
print("loss: ", loss) 


In [ ]:
class ProduceExample(tf.keras.callbacks.Callback): 
    def __init__(self, dataset) -> None: 
        self.dataset = dataset.as_numpy_iterator()
    
    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.dataset.next()
        yhat = self.model.predict(data[0])
        decoded = tf.keras.backend.ctc_decode(yhat, [75,75], greedy=False)[0][0].numpy()
        for x in range(len(yhat)):           
            print('Original:', tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_to_char(decoded[x])).numpy().decode('utf-8'))
            print('~'*100)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)

In [ ]:
# checkpoint_callback = ModelCheckpoint(os.path.join('models','checkpoint_with_attention'), monitor='loss', save_weights_only=True, verbose = 1) 

In [ ]:
# checkpoint_callback = ModelCheckpoint(os.path.join('data','models','checkpoint','model1_checkpoint_1-100.keras'), monitor='loss', save_weights_only = False, verbose = 1) 

checkpoint_callback = ModelCheckpoint(os.path.join(WORKING_DIR,'models','Checkpoints','with_attention','model_checkpoint.keras'), monitor='loss', save_weights_only = False, verbose = 1) 

In [ ]:
save_weights = ModelCheckpoint(os.path.join(WORKING_DIR,'models','Weights','model_with_attention_weights.weights.h5'), monitor='loss', save_weights_only = True, verbose = 1) 

In [ ]:
schedule_callback = LearningRateScheduler(scheduler)

In [ ]:

example_callback = ProduceExample(test)

In [ ]:
model.fit(train, epochs=50, callbacks=[checkpoint_callback, schedule_callback, example_callback, save_weights])

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/models/Checkpoints/with_attention/model_checkpoint.keras')

In [ ]:
model.fit(train, epochs=50, initial_epoch = 30, callbacks=[checkpoint_callback, schedule_callback, example_callback, save_weights])

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/models/Checkpoints/with_attention/model_checkpoint.keras')

In [ ]:
model.compile(optimizer=optimizer, loss=CTCLoss)

In [ ]:
#changes in optimizers and scheduler after 50 epochs

from tensorflow.keras.callbacks import EarlyStopping

optimizer = Adam(
    learning_rate=1e-4,  # Lower initial LR for stable CTC convergence
    beta_1=0.9,
    beta_2=0.999,
    clipnorm=1.0  # Prevent gradient explosion
)

# Improved scheduler (linear warmup + plateau-based decay)
def lr_scheduler(epoch, lr):
    if epoch < 60:  # Warmup phase
        return lr * 1.2
    elif (epoch+1) % 20 == 0:  # Step decay
        return lr * 0.5
    else:
        return lr

early_stopping = EarlyStopping(
    monitor="loss",  # CTC loss on validation set
    patience=12,         # Wait 12 epochs without improvement
    min_delta=0.005,     # Changes < 0.005 are considered noise
    restore_best_weights=True,  # Critical for deployment
    verbose=1
)

checkpoint_callback = ModelCheckpoint(os.path.join(WORKING_DIR,'models','Checkpoints','with_attention','model_checkpoint_after_50.keras'), 
                                      monitor='loss', 
                                      save_weights_only = False, 
                                      verbose = 1) 

class SaveWeightsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        filename = "/kaggle/working/models/Weights/model_with_attention_weights_after_50.weights.h5"
        self.model.save_weights(filename)
        print(f"Saved weights at {filename}")

save_weights = SaveWeightsCallback()

schedule_callback = LearningRateScheduler(lr_scheduler)

example_callback = ProduceExample(test)

In [ ]:
model.fit(train, epochs=100, initial_epoch = 50, callbacks=[checkpoint_callback, schedule_callback, example_callback, save_weights, early_stopping])

# 5. Make a Prediction 

In [ ]:
url = 'https://drive.google.com/uc?id=1vWscXs4Vt0a_1IH1-ct2TCgXAZT-N3_Y'
output = 'checkpoints.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('checkpoints.zip', 'models')

In [ ]:
model.load_weights('models/checkpoint')

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
sample = test_data.next()

In [ ]:
yhat = model.predict(sample[0])

In [ ]:
print('~'*100, 'REAL TEXT')
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in sample[1]]

In [ ]:
decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75,75], greedy=False)[0][0].numpy()

In [ ]:
print('~'*100, 'PREDICTIONS')
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]

# Test on a Video

In [ ]:
model.load_weights('/kaggle/working/models/Weights/model_with_attention_weights_after_50.weights.h5')

In [ ]:
sample = load_data(tf.convert_to_tensor('/kaggle/working/data/s1/bbbs5s.mpg'))

In [ ]:
sample[0].shape

In [ ]:
print('~'*100, 'REAL TEXT')
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in [sample[1]]]

In [ ]:
yhat = model.predict(tf.expand_dims(sample[0], axis=0))

In [ ]:
yhat[0].shape

In [ ]:
decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75], greedy=True)[0][0].numpy()

In [ ]:
print('~'*100, 'PREDICTIONS')
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]

In [ ]:
model.load_weights('/kaggle/working/models/Weights/model_with_attention_weights_after_50.weights.h5')

In [ ]:
# Function to reset states for all LSTM layers
def reset_lstm_states(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Bidirectional):
            layer.reset_states()


In [ ]:
#testing videos ly passing 15 frames of a video at a time instead of 75 

def load_fixed_segments(path:str):
    frames = load_video(path)

    segments = np.split(frames, 3)

    return np.array(segments)

def predict(video_path):
    segments = load_fixed_segments(video_path)

    all_preds = []
    for segment in segments:
        segment = np.expand_dims(segment, axis = 0)
        yhat = model.predict(segment)
        decoded = tf.keras.backend.ctc_decode(yhat, input_length=[25], greedy=True)[0][0].numpy()
        text = tf.strings.reduce_join(num_to_char(decoded[0])).numpy().decode('utf-8')
        all_preds.append(text)

    reset_lstm_states(model)
    
    final_prediction = " ".join(all_preds)
    return final_prediction

In [ ]:
video_path = '/kaggle/working/data/s1/bbbs6p.mpg'
alignments_path = "/kaggle/working/data/alignments/s1/bbbs6p.align"
y_true = load_alignments(alignments_path)
real = tf.strings.reduce_join([num_to_char(word) for word in y_true]).numpy().decode('utf-8') 
print(f"Real Allignments: {real}")
pred = predict(video_path)
print(f"Predicted allignments: {pred}")


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os

# Load trained model
# model = tf.keras.models.load_model("path_to_your_model.h5", custom_objects={"CTCLoss": CTCLoss})

# Call this function before processing a new video

# Function to process video in 15-frame windows
def load_video_sliding_window(path: str, window_size=25, stride=25):
    frames = load_video(path)
    # Extract 15-frame windows
    windows = []
    for start in range(0, len(frames) - window_size + 1, stride):
        windows.append(frames[start:start + window_size])

    return np.array(windows)  # Shape: (num_windows, 15, 46, 140, 1)

# Function to predict on a single video using 15-frame windows
def predict_video(video_path):
    windows = load_video_sliding_window(video_path)

    all_predictions = []
    for window in windows:
        window = np.expand_dims(window, axis=0)  # Add batch dimension
        prediction = model.predict(window)
        decoded = tf.keras.backend.ctc_decode(prediction, [25], greedy=True)[0][0].numpy()
        text = tf.strings.reduce_join(num_to_char(decoded[0])).numpy().decode('utf-8')
        all_predictions.append(text)

    reset_lstm_states(model)
    # Combine predictions from all windows
    final_prediction = " ".join(all_predictions)
    return final_prediction

# Example Usage
video_path = '/kaggle/working/data/s1/bbbs6p.mpg'
alignments_path = "/kaggle/working/data/alignments/s1/bbbs6p.align"
y_true = load_alignments(alignments_path)
real = tf.strings.reduce_join([num_to_char(word) for word in y_true]).numpy().decode('utf-8') 
print(f"Real Allignments: {real}")
predicted_text = predict_video(video_path)
print(f"Predicted text: {predicted_text}")


In [ ]:
def lr_scheduler(epoch, lr):
    if epoch < 60:  # Warmup phase
        return lr * 1.2
    elif (epoch+1) % 20 == 0:  # Step decay
        return lr * 0.5
    else:
        return lr

In [ ]:
frames.shape

In [ ]:
model.load_weights('/kaggle/working/models/Weights/model_with_attention_weights_after_50.weights.h5')

In [ ]:

import json

# Define checkpoint directory
DIR = WORKING_DIR + "/models/state"
os.makedirs(DIR, exist_ok=True)

def save_training_state(model, optimizer, epoch):
    """
    Saves model weights, optimizer state, and training configs.
    """
    # Save model weights
    model.save_weights(os.path.join(DIR, "model_f25.weights.h5"))
    
     # Save state
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    checkpoint.save(os.path.join(DIR, "checkpoint"))
    
    # Save epoch info
    with open(os.path.join(DIR, "epoch.txt"), "w") as f:
        f.write(str(epoch))

    print(f"Model state saved at epoch {epoch+1}\n")


In [ ]:
DIR = WORKING_DIR + "/models/state"
epoch = 39
with open(os.path.join(DIR, "epoch.txt"), "w") as f:
        f.write(str(epoch))

['40']


In [ ]:
save_training_state(model, optimizer, 0)

In [ ]:


def load_training_state(model, optimizer):
    """
    Loads model weights, optimizer state, and training configs.
    """
    try:
        # Load model weights
        model.load_weights(os.path.join(DIR, "model_f25.weights.h5"))
        print("Loaded model weights.")
        
        checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
        
        checkpoint.restore(tf.train.latest_checkpoint(DIR))
        print("Loaded model and optimizer state.")

        # Load last saved epoch
        with open(os.path.join(DIR, "epoch.txt"), "r") as f:
            start_epoch = int(f.read()) + 1   # Resume from next epoch
        print(f"Resuming training from epoch {start_epoch + 1}")

    except Exception as e:
        print("No checkpoint found. Starting from scratch.")
        start_epoch = 0  # Start from the beginning

    return start_epoch


In [ ]:
st = load_training_state(model, optimizer)
print(st)

In [ ]:
def CTCLoss_single_video(y_true, y_pred):
    
    #y_true = tf.squeeze(y_true, axis=0)  # Remove batch dimension (if any)
    # print(y_true.shape)
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int32")
    
    input_length = tf.shape(y_pred)[1]  # Total time steps in logits
    label_length = tf.shape(y_true)[1]  # Length of the ground truth sequence

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int32")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int32")

    # input_length = tf.expand_dims(input_length, axis=0)  # Make it a scalar tensor
    # label_length = tf.expand_dims(label_length, axis=0)  # Make it a scalar tensor

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


In [ ]:
frames = frames[0]
frames = tf.expand_dims(frames,axis = 0)
y_true = alignments[0]
y_pred = model.predict(frames)
print("Before Passing to ctcloss")
print("Alignments shape : ", y_true.shape)
print("Predictions shape : ", y_pred.shape)
loss = CTCLoss_single_video(y_true, y_pred)
print(loss)

In [ ]:
optimizer = Adam(
    learning_rate=1e-4,  # Lower initial LR for stable CTC convergence
    beta_1=0.9,
    beta_2=0.999,
    clipnorm=1.0  # Prevent gradient explosion
)

# lr = 1e-5
# print(optimizer.learning_rate.numpy())
# optimizer.learning_rate.assign(lr)
# print(optimizer.learning_rate.numpy())

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch < 15:  # Warmup phase
        return lr * 1.2
    elif (epoch+1) % 20 == 0:  # Step decay
        return lr * 0.5
    else:
        return lr

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_loss = float('inf')
        self.wait = 0

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.wait = 0  # Reset counter
        else:
            self.wait += 1  # Increment counter

        if self.wait >= self.patience:
            print("\nEarly stopping triggered.")
            return True  # Stop training
        return False

In [ ]:
class ProduceExample: 
    def __init__(self, dataset):
        self.dataset = dataset.as_numpy_iterator()
    
    def produce(self, model):
        decode = []
        frames, alignments = self.dataset.next()
        batch_size =  frames.shape[0]
        for i in range(batch_size):
            frames_i = frames[i]  # Get video frames
            alignment = alignments[i]  # Get ground truth alignment
            # video_length = batch_video_lengths[i]  # Total frames in video
    
            # Split video into chunks
            chunks = split_video_into_chunks(frames_i)
    
            # Reset LSTM states before processing each video
            for layer in model.layers:
                if isinstance(layer, tf.keras.layers.Bidirectional):
                    layer.reset_states()
    
            all_logits = []
            input_lengths = []
    
            for chunk in chunks:
                chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
                logits = model(chunk, training=True)  # Forward pass
                all_logits.append(logits)
                input_lengths.append(logits.shape[1])  # Time steps in logits

            # Combine chunks into one continuous sequence
            all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)
            input_length = tf.convert_to_tensor([sum(input_lengths)], dtype=tf.int32)
            
            decoded = tf.keras.backend.ctc_decode(all_logits, input_length, greedy=False)[0][0].numpy()
            decode.append(decoded)
         
        for x in range(len(decode)):           
            print('Original:', tf.strings.reduce_join(num_to_char(alignments[x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_to_char(decode[x])).numpy().decode('utf-8'))
            print('~'*100)

In [ ]:
# def produceExample(model, dataset):
#     decode = []
#     frames, alignments = dataset.as_numpy_iterator().next()
#     batch_size =  frames.shape[0]
#     for i in range(batch_size):
#         frames_i = frames[i]  # Get video frames
#         alignment = alignments[i]  # Get ground truth alignment
#         # video_length = batch_video_lengths[i]  # Total frames in video

#         # Split video into chunks
#         chunks = split_video_into_chunks(frames_i)

#         # Reset LSTM states before processing each video
#         for layer in model.layers:
#             if isinstance(layer, tf.keras.layers.Bidirectional):
#                 layer.reset_states()

#         all_logits = []
#         input_lengths = []

#         for chunk in chunks:
#             chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
#             logits = model(chunk, training=True)  # Forward pass
#             all_logits.append(logits)
#             input_lengths.append(logits.shape[1])  # Time steps in logits

#         # Combine chunks into one continuous sequence
#         all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)
#         input_length = tf.convert_to_tensor([sum(input_lengths)], dtype=tf.int32)
        
#         decoded = tf.keras.backend.ctc_decode(all_logits, input_length, greedy=False)[0][0].numpy()
#         decode.append(decoded)
     
#     for x in range(len(decode)):           
#         print('Original:', tf.strings.reduce_join(num_to_char(alignments[x])).numpy().decode('utf-8'))
#         print('Prediction:', tf.strings.reduce_join(num_to_char(decode[x])).numpy().decode('utf-8'))
#         print('~'*100)

In [ ]:
example = ProduceExample(test)

example.produce(model)

In [ ]:
# def split_video_into_chunks(frames, frames_per_chunk=25):

#     frames = tf.convert_to_tensor(frames)  # Ensure TensorFlow tensor

#     def numpy_split(frames_np):
#         num_chunks = len(frames_np) // frames_per_chunk
#         chunks = np.array_split(frames_np[:num_chunks * frames_per_chunk], num_chunks)
#         return [chunk.astype(np.float32) for chunk in chunks]  # Convert to float32

#     chunks = tf.numpy_function(numpy_split, [frames], [tf.float32] * frames_per_chunk)
#     return chunks

In [15]:
def split_video_into_chunks(frames, frames_per_chunk=25):
    """
    Splits the full video into fixed-size chunks.
    Returns a list of frame chunks.
    """
    total_frames = len(frames)
    num_chunks = total_frames // frames_per_chunk

    # Split into fixed-size chunks
    chunks = np.array_split(frames[:num_chunks * frames_per_chunk], num_chunks)

    return np.array(chunks)

In [ ]:
!pip install tqdm

In [ ]:

def train_step(model, optimizer, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]  # Number of videos in the batch
    total_loss = 0.0  # Accumulate batch loss

    with tf.GradientTape() as tape:
        
        for i in range(batch_size):
            frames = batch_frames[i]  # Get video frames
            alignment = batch_alignments[i]  # Get ground truth alignment
            # video_length = batch_video_lengths[i]  # Total frames in video

            # Split video into chunks
            chunks = split_video_into_chunks(frames, frames_per_chunk)

            # Reset LSTM states before processing each video
            for layer in model.layers:
                if isinstance(layer, tf.keras.layers.Bidirectional):
                    layer.reset_states()

            all_logits = []
            input_lengths = []

            for chunk in chunks:
                chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
                logits = model(chunk, training=True)  # Forward pass
                all_logits.append(logits)
                input_lengths.append(logits.shape[1])  # Time steps in logits

            # Combine chunks into one continuous sequence
            all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)
            input_length = tf.reduce_sum(input_lengths) * tf.ones(shape=(1, 1), dtype="int32")

            # Compute loss for this video
            loss = CTCLoss_single_video(alignment, all_logits)
            total_loss += loss  # Accumulate loss

        # Compute batch loss
        batch_loss = total_loss / batch_size  # Average loss across batch

    # Compute gradients and update weights
    grads = tape.gradient(batch_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Update model weights

    return batch_loss 

def train_model(model, train_dataset, test_dataset, num_batches, optimizer, epochs=50):

    example = ProduceExample(test_dataset)
    early_stopping = EarlyStopping()
    start_epoch = load_training_state(model, optimizer)
    
    for epoch in range(start_epoch, epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        lr = lr_scheduler(epoch, optimizer.learning_rate.numpy())
        optimizer.learning_rate.assign(lr)
        print(f"Current Learning Rate: {lr:.6f}")

        total_epoch_loss = 0.0
        # Number of batches per epoch
        for batch_idx in tqdm(range(num_batches)):
            batch_frames, batch_alignments = train_dataset.as_numpy_iterator().next()
            loss = train_step(model, optimizer, batch_frames, batch_alignments)
            total_epoch_loss += loss.numpy().item()
            
        avg_epoch_loss = total_epoch_loss / num_batches  # Compute average loss
        print(f"Epoch {epoch+1} Completed. Average Loss: {avg_epoch_loss:.4f}\n")

        if early_stopping.step(avg_epoch_loss):
            break

        if early_stopping.wait == 0:
            save_training_state(model, optimizer, epoch)

        example.produce(model)


In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 20)

The model's loss began to increase after 9 epoch and early stopping was triggered, so model contains weights and state till epoch 9. Now, continuing from 10th epoch onwards, also doing some minor changes in lr scheduler


In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 20)

training stopped unexpectedly so starting from where left off

In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 20)

In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 30)

In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 40)

In [ ]:
train_model(model, train, test, train_batch, optimizer, epochs = 41)

In [ ]:
# updated lr scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:  # Warmup phase updated from 15 to 10
        return lr * 1.2
    elif (epoch+1) % 20 == 0:  # Step decay
        return lr * 0.5
    else:
        return lr

In [ ]:
loss = train_step(model, optimizer, frames, alignments)
print(loss)

In [ ]:
# import shutil 

# shutil.rmtree('/kaggle/working/models/state')

In [21]:
def CTCLoss_single_video(y_true, y_pred):
    
    #y_true = tf.squeeze(y_true, axis=0)  # Remove batch dimension (if any)
    # print(y_true.shape)
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int32")
    
    input_length = tf.shape(y_pred)[1]  # Total time steps in logits
    label_length = tf.shape(y_true)[1]  # Length of the ground truth sequence

    # input_length = tf.convert_to_tensor([input_length], dtype=tf.int32)  # Shape (1,)
    # label_length = tf.convert_to_tensor([label_length], dtype=tf.int32)  # Shape (1,)

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int32")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int32")

    # tf.print("y_true shape:", tf.shape(y_true))
    # tf.print("y_pred shape:", tf.shape(y_pred))
    # tf.print("input_length:", input_length)
    # tf.print("label_length:", label_length)

    # input_length = tf.expand_dims(input_length, axis=0)  # Make it a scalar tensor
    # label_length = tf.expand_dims(label_length, axis=0)  # Make it a scalar tensor

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


In [16]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.5 MB/s eta 0:00:0000:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [52]:
from jiwer import wer,mer

def test_step(model, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]
    # input_length = batch_frames.shape[1]
    total_loss = 0.0
    # wer_i, ser_i = 0.0, 0.0

    for i in range(batch_size):
        frames_i = batch_frames[i]
        alignment = batch_alignments[i]
        # og = tf.strings.reduce_join([num_to_char(word) for word in alignment[0]]).numpy().decode('utf-8')

        chunks = split_video_into_chunks(frames_i, frames_per_chunk)

        # Reset LSTM states before processing each video
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Bidirectional):
                layer.reset_states()

        all_logits = []
        # input_lengths = []
        
        for chunk in chunks:
            chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
            # print(chunk.shape)
            logits = model.predict(chunk, verbose = 0)  # Training is False since it is used for testing and not for forward pass or training
            all_logits.append(logits)
            # input_lengths.append(logits.shape[1])  # Time steps in logits

        # Combine chunks into one continuous sequence
        all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)

        # decoded = tf.keras.backend.ctc_decode(all_logits, input_length=[input_length], greedy=True)[0][0].numpy()
        # predicted = tf.strings.reduce_join([num_to_char(word) for word in decoded[0]]).numpy().decode('utf-8')

        # wer_score = wer(og, predicted)
        # ser_score = mer(og, predicted)

        # wer_i += wer_score
        # ser_i += ser_score

        # print("Allignment before passing to ctcloss: ", alignment)
        # Compute loss for this video
        loss = CTCLoss_single_video(alignment, all_logits)
        total_loss += loss  # Accumulate loss

    batch_loss = total_loss / batch_size
    # batch_wer = wer_i / batch_size
    # batch_ser = ser_i / batch_size
    # print("done")

    return batch_loss
        

In [53]:


DIR = WORKING_DIR + "/models/state"

def test_model(model, dataset, num_batches):

    # dataset = iter(dataset)
    
    model.load_weights(os.path.join(DIR, "model_f25.weights.h5"))
    print("Loaded model weights.")

    total_loss = 0.0
    # wer_score, ser_score = 0.0, 0.0

    for batch_idx in tqdm(range(num_batches)):
        d = dataset.next()
        frames = d[0]
        alignments = d[1]
        # print(alignments.shape)
        loss = test_step(model, frames, alignments)
        total_loss += loss
        # wer_score += batch_wer
        # ser_score += batch_ser

    avg_loss = total_loss / num_batches
    # avg_wer = wer_score / num_batches
    # avg_ser = ser_score / num_batches

    print(f"Testing Completed. Average Loss: {float(avg_loss):.4f}\n")
    # print(f"Word Error Rate: {avg_wer:.4f}\n")
    # print(f"Sentence Error Rate: {avg_ser:.4f}\n")

In [17]:
 test_dataset = test.as_numpy_iterator()

In [54]:
test_model(model, test_dataset, test_batch)

Loaded model weights.


  0%|          | 0/50 [00:00<?, ?it/s]

Testing Completed. Average Loss: 0.2317



In [23]:
train_dataset = train.as_numpy_iterator()

In [56]:
test_model(model, train_dataset, train_batch)

Loaded model weights.


  0%|          | 0/450 [00:00<?, ?it/s]

Testing Completed. Average Loss: 0.1676



In [18]:
from jiwer import wer,mer

def test_step(model, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]
    input_length = batch_frames.shape[1]
    total_loss = 0.0
    wer_i, ser_i = 0.0, 0.0

    for i in range(batch_size):
        frames_i = batch_frames[i]
        alignment = batch_alignments[i]
        og = tf.strings.reduce_join([num_to_char(word) for word in alignment[0]]).numpy().decode('utf-8')

        chunks = split_video_into_chunks(frames_i, frames_per_chunk)

        # Reset LSTM states before processing each video
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Bidirectional):
                layer.reset_states()

        all_logits = []
        # input_lengths = []
        
        for chunk in chunks:
            chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
            # print(chunk.shape)
            logits = model.predict(chunk, verbose = 0)  # Training is False since it is used for testing and not for forward pass or training
            all_logits.append(logits)
            # input_lengths.append(logits.shape[1])  # Time steps in logits

        # Combine chunks into one continuous sequence
        all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)

        decoded = tf.keras.backend.ctc_decode(all_logits, input_length=[input_length], greedy=True)[0][0].numpy()
        predicted = tf.strings.reduce_join([num_to_char(word) for word in decoded[0]]).numpy().decode('utf-8')

        wer_score = wer(og, predicted)
        ser_score = mer(og, predicted)

        wer_i += wer_score
        ser_i += ser_score

        # print("Allignment before passing to ctcloss: ", alignment)
        # Compute loss for this video
        loss = CTCLoss_single_video(alignment, all_logits)
        total_loss += loss  # Accumulate loss

    batch_loss = total_loss / batch_size
    batch_wer = wer_i / batch_size
    batch_ser = ser_i / batch_size
    # print("done")

    return batch_loss, batch_wer, batch_ser
        

In [19]:


DIR = WORKING_DIR + "/models/state"

def test_model(model, dataset, num_batches):

    # dataset = iter(dataset)
    
    model.load_weights(os.path.join(DIR, "model_f25.weights.h5"))
    print("Loaded model weights.")

    total_loss = 0.0
    wer_score, ser_score = 0.0, 0.0

    for batch_idx in tqdm(range(num_batches)):
        d = dataset.next()
        frames = d[0]
        alignments = d[1]
        # print(alignments.shape)
        loss, batch_wer, batch_ser = test_step(model, frames, alignments)
        total_loss += loss
        wer_score += batch_wer
        ser_score += batch_ser

    avg_loss = total_loss / num_batches
    avg_wer = wer_score / num_batches
    avg_ser = ser_score / num_batches

    print(f"Testing Completed. Average Loss: {float(avg_loss):.4f}\n")
    print(f"Word Error Rate: {avg_wer:.4f}\n")
    print(f"Sentence Error Rate: {avg_ser:.4f}\n")

In [22]:
test_model(model, test_dataset, test_batch)

Loaded model weights.


  0%|          | 0/50 [00:00<?, ?it/s]

Testing Completed. Average Loss: 0.0824

Word Error Rate: 0.0029

Sentence Error Rate: 0.0029



In [24]:
test_model(model, train_dataset, train_batch)

Loaded model weights.


  0%|          | 0/450 [00:00<?, ?it/s]

Testing Completed. Average Loss: 0.1894

Word Error Rate: 0.0015

Sentence Error Rate: 0.0015



In [29]:
dataset_iter = iter(test)

frames, alignments = next(dataset_iter)

In [48]:
print(frames[0].shape)
print(alignments.shape[0])

2
2


In [16]:
# testing on a video

path = tf.convert_to_tensor('/kaggle/working/data/s1/bbas1s.mpg')

frames, alignments = load_data(path)

In [17]:
frames.shape

(75, 46, 140, 1)

In [17]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [18]:
from jiwer import wer,mer

chunks = split_video_into_chunks(frames)

DIR = WORKING_DIR + "/models/state"

model.load_weights(os.path.join(DIR, "model_f25.weights.h5"))
print("Loaded model weights.")

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Bidirectional):
        layer.reset_states()

all_logits = []

for chunk in chunks:
    chunk = tf.expand_dims(chunk, axis = 0)
    logits = model.predict(chunk, verbose = 0)  # Training is False since it is used for testing and not for forward pass or training
    decoded_i = tf.keras.backend.ctc_decode(logits, input_length=[25], greedy=True)[0][0].numpy()
    predicted_i = tf.strings.reduce_join([num_to_char(word) for word in decoded_i[0]]).numpy().decode('utf-8')
    print("Predicted: ", predicted_i)
    all_logits.append(logits)
    
all_logits = tf.concat(all_logits, axis=1)

decoded = tf.keras.backend.ctc_decode(all_logits, input_length=[75], greedy=True)[0][0].numpy()

predicted = tf.strings.reduce_join([num_to_char(word) for word in decoded[0]]).numpy().decode('utf-8')

og = tf.strings.reduce_join([num_to_char(word) for word in alignments[0]]).numpy().decode('utf-8')

wer_score = wer(og, predicted)
ser_score = mer(og, predicted)

print("Original: ", og)
print("Predicted: ", predicted)

print(f"Word Error Rate (WER): {wer_score:.4f}")
print(f"Sentence Error Rate (SER): {ser_score:.4f}")

Loaded model weights.
Predicted:  bin blue at s 
Predicted:  one soon
Predicted:  
Original:  bin blue at s one soon
Predicted:  bin blue at s one soon
Word Error Rate (WER): 0.0000
Sentence Error Rate (SER): 0.0000


In [21]:
os.remove('/kaggle/working/S2-10.zip')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/S2-10.zip'

In [ ]:
https://drive.google.com/file/d/12jGPlN2B4hdT57CA3ZahdCXpaRyHdhk-/view?usp=sharing

In [33]:
# import gdown

# url = 'https://drive.google.com/uc?id=12jGPlN2B4hdT57CA3ZahdCXpaRyHdhk-'
# output = 'S2-10.zip'
# gdown.download(url, output, quiet=False)
# gdown.extractall('S2-10.zip', 'S2-10')


In [30]:
os.mkdir('S2-10')

In [31]:
import shutil
shutil.rmtree('/kaggle/working/S2-10')
os.remove('/kaggle/working/S2-10.zip')

In [26]:
shutil.move('/kaggle/working/S2-10/align', '/kaggle/working/S2-10/s4')

'/kaggle/working/S2-10/s4/align'

In [31]:
print(os.listdir(WORKING_DIR))

['S2-10', 'S6', 'checkpoints.zip', '.virtual_documents', 'models', 'S9', 'S8', '.ipynb_checkpoints', 'data.zip', 'S5', 'S7', 'animation.gif', 'S2', 'S3', 'data', 'S10', 'S4', 'S2-10.zip']


In [25]:
import shutil

ss = ['S2','S3','S4','S5','S6','S7','S8','S9','S10']

for folder in os.listdir(WORKING_DIR):
    if folder in ss:
        source = os.path.join(WORKING_DIR, folder)
        destination = os.path.join(WORKING_DIR, 'S2-10')
        shutil.move(source, destination)
        print(f'Moved folder {folder} to "S2-10"')

Moved folder S4 to "S2-10"
Moved folder S9 to "S2-10"
Moved folder S7 to "S2-10"
Moved folder S2 to "S2-10"
Moved folder S10 to "S2-10"
Moved folder S6 to "S2-10"
Moved folder S5 to "S2-10"
Moved folder S3 to "S2-10"
Moved folder S8 to "S2-10"


In [30]:
for video in os.listdir('/kaggle/working/S2-10/S4'):
    if video != 'align':
        shutil.move(f'/kaggle/working/S2-10/S4/{video}', f'/kaggle/working/S2-10/S4/s4/')
        

FileNotFoundError: Directory does not exist: /kaggle/working/S2-10/S4/s4/

In [27]:
old = '/kaggle/working/S2-10/s4'
new = '/kaggle/working/S2-10/S4'

os.rename(old, new)

In [22]:
shutil.rmtree('/kaggle/working/set1')
shutil.rmtree('/kaggle/working/set2')

In [34]:
os.makedirs('set1/Alignments',exist_ok = True)
os.makedirs('set1/Videos',exist_ok = True)
os.makedirs('set2/Alignments',exist_ok = True)
os.makedirs('set2/Videos',exist_ok = True)

In [44]:
for file in os.listdir(os.path.join(WORKING_DIR, 'S2-10', 'S2', 'align')):
    print(file)
    break

lriy1p.align


In [35]:
set1 = ['S2','S3','S4','S5']
set2 = ['S6','S7','S8','S9','S10']

for ss in os.listdir(os.path.join(WORKING_DIR,'S2-10')):
    if ss in set1:
        for folder in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss)):
            if folder == 'align':
                for file in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss, folder)):
                    source = os.path.join(WORKING_DIR, 'S2-10', ss, folder, file)
                    destination = os.path.join(WORKING_DIR, 'set1', 'Alignments')
                    shutil.copy(source, destination)
            else:
                for file in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss, ss.lower())):
                    source = os.path.join(WORKING_DIR, 'S2-10', ss, ss.lower(), file)
                    destination = os.path.join(WORKING_DIR, 'set1', 'Videos')
                    shutil.copy(source, destination)
        print(f"Moved alignments and videos of {ss} to set1")
    if ss in set2:
        for folder in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss)):
            if folder == 'align':
                for file in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss, folder)):
                    source = os.path.join(WORKING_DIR, 'S2-10', ss, folder, file)
                    destination = os.path.join(WORKING_DIR, 'set2', 'Alignments')
                    shutil.copy(source, destination)
            else:
                for file in os.listdir(os.path.join(WORKING_DIR, 'S2-10', ss, ss.lower())):
                    source = os.path.join(WORKING_DIR, 'S2-10', ss, ss.lower(), file)
                    destination = os.path.join(WORKING_DIR, 'set2', 'Videos')
                    shutil.copy(source, destination)
        print(f"Moved alignments and videos of {ss} to set2")

Moved alignments and videos of S4 to set1
Moved alignments and videos of S9 to set2
Moved alignments and videos of S7 to set2
Moved alignments and videos of S2 to set1
Moved alignments and videos of S10 to set2
Moved alignments and videos of S6 to set2
Moved alignments and videos of S5 to set1
Moved alignments and videos of S3 to set1
Moved alignments and videos of S8 to set2


3997

In [14]:
corrupted = []

In [37]:
import subprocess

# check if the video is corrupted or not
def is_video_corrupted(path: str) -> bool:
    # FFmpeg command to detect video errors
    command = ["ffmpeg", "-v", "error", "-i", path, "-f", "null", "-"]
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.stderr:
            print(f"Errors found in video {path}:\n{result.stderr}")
            return True
    except Exception as e:
        print(f"Error running FFmpeg: {e}")
        return True
    return False

def get_corrupted(chunk:str):
    for file in os.listdir(os.path.join(WORKING_DIR, chunk, 'Videos')):
        #print(file)
        path = os.path.join(WORKING_DIR, chunk, 'Videos',f'{file}')
        # print(f"Processing file: {file}")
        if is_video_corrupted(path):
            corrupted.append(file)
            
get_corrupted("set1")

Errors found in video /kaggle/working/set1/Videos/Thumbs.db:
/kaggle/working/set1/Videos/Thumbs.db: Invalid data found when processing input

Errors found in video /kaggle/working/set1/Videos/sran6s.mpg:
[mpeg1video @ 0x57b152e25800] ac-tex damaged at 21 17
[mpeg1video @ 0x57b152e25800] Warning MVs not available

Errors found in video /kaggle/working/set1/Videos/lgil4a.mpg:
[mpeg1video @ 0x5cf828ccb800] ac-tex damaged at 22 17
[mpeg1video @ 0x5cf828ccb800] Warning MVs not available



In [38]:
print(corrupted)

['Thumbs.db', 'Thumbs.db', 'sran6s.mpg', 'lgil4a.mpg']


In [6]:
print(corrupted)

['lgil4a.mpg', 'Thumbs.db', 'sran6s.mpg']


[]


In [42]:
# removing corruped files in set1

for file in corrupted[2:]:
    if(file.endswith('.mpg')):
        os.remove(os.path.join(WORKING_DIR,'set1', 'Videos',f'{file}'))
        os.remove(os.path.join(WORKING_DIR,'set1', 'Alignments', f"{file.split('.')[0]}.align"))
    else:
        os.remove(os.path.join(WORKING_DIR,'set1', 'Videos',f'{file}'))

In [4]:
import os
os.remove('data.zip')

In [2]:
import gdown

gdown.extractall('data.zip')

BadZipFile: File is not a zip file

In [7]:
# url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
# output = os.path.join(WORKING_DIR,'data.zip')
# gdown.download(url, output, quiet=False)
# gdown.extractall('data.zip')

In [9]:
aligns = []

s1 = '/kaggle/working/data/alignments/s1'

for align in os.listdir(s1):
    with open(os.path.join(s1,align), 'r') as file:
        lines = file.readlines()
        
    text = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            text.append(line[2])

    aligns.append(' '.join(text)) 

set1 = '/kaggle/working/set1/Alignments'

for align in os.listdir(set1):
    with open(os.path.join(set1,align), 'r') as file:
        lines = file.readlines()
        
    text = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            text.append(line[2])

    aligns.append(' '.join(text)) 

In [10]:
len(aligns)

4998

In [ ]:
alignments_data = {
    "alignments ": aligns
}

# Write to file
with open("alignments.json", "w") as file:
    json.dump(alignments_data, file, indent=4)

In [13]:
import pickle

# Save corpus words as pickle
with open("alignments.pkl", "wb") as f:
    pickle.dump(aligns, f)

In [10]:
import pickle

with open("alignments.pkl", "rb") as f:
    aligns = pickle.load(f)

print(aligns[0])

place green by r two please


In [5]:
import os 

WORKING_DIR = os.getcwd()

print("Working Directory:", WORKING_DIR)

import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
from tqdm.notebook import tqdm

Working Directory: /kaggle/working


In [3]:
def count_what(file: str, entity:str):
    count = 0
    if entity == 'videos':
        for video in os.listdir(os.path.join(f'{file}/Videos')):
            count += 1
    elif entity == 'allignments':
        for allign in os.listdir(os.path.join(f'{file}/Alignments')):
            count += 1
    else:
        print("Select between videos and allignments!")
        return
            
    return count

count_what('set1','videos')

3997

In [4]:
def load_video(path:str) -> List[float]: 

    # print(f'Processing video: {path.split("/")[-1]}')
    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))): 
        ret, frame = cap.read()
        if not ret or frame is None:
            print(f"Warning: Skipping empty or unreadable frame for video: {path.split('/')[-1]}")
            continue
        frame = tf.image.rgb_to_grayscale(frame)
        # print(frame.shape)
        frames_i = frame[190:236,80:220,:]
        # print(frames_i.shape)
        frames.append(frames_i)
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # roi = frame[190:236,80:220]
        # # frame = cv2.resize(roi, (140, 46))
        # frame_i = tf.expand_dims(roi, axis = -1)
        # frames.append(frame_i)
    cap.release()

    frames = np.array(frames)

    # Normalize frames
    mean = np.mean(frames)
    std = np.std(frames) + 1e-8
    normalized_frames = (frames - mean) / (std)  # Prevent divide by zero
    return normalized_frames
    
    # mean = tf.math.reduce_mean(frames)
    # std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    # return tf.cast((frames - mean), tf.float32) / std

vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil': 
            tokens = [*tokens,' ',line[2]]
    # temp = tf.strings.unicode_split(tokens, input_encoding='UTF-8')
    # print(temp)
    # print(temp.shape)
    # temp = tf.reshape(tf.strings.unicode_split(temp, input_encoding='UTF-8'), (-1))
    # print(temp)
    # print(temp.shape)
    result = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]
    # print(result.shape)
    result = tf.expand_dims(result, axis = 0)
    return result

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '] (size =40)


In [17]:
fram = load_video('/kaggle/working/set1/Videos/briz6a.mpg')
len(fram)

Processing video: briz6a.mpg


75

In [45]:
# function to get videos containing corrupted or unreadable frames

for video in os.listdir('/kaggle/working/set1/Videos'):
    load_video(os.path.join('/kaggle/working/set1/Videos',video))

In [46]:
os.remove('/kaggle/working/set1/Videos/lwirzs.mpg')

In [5]:
def load_data(path: str): 
    path = bytes.decode(path.numpy())
    file_name = path.split('/')[-1].split('.')[0]
    # File name splitting for windows
    # file_name = path.split('\\')[-1].split('.')[0]
    video_path = os.path.join('set1','Videos',f'{file_name}.mpg')
    alignment_path = os.path.join('set1','Alignments',f'{file_name}.align')
    frames = load_video(video_path) 
    alignments = load_alignments(alignment_path)
    
    return frames, alignments

def mappable_function(path:str) ->List[str]:
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

In [18]:
path = tf.convert_to_tensor('/kaggle/working/set1/Videos/sgbc5n.mpg')
f, a = load_data(path)
print(a.numpy())

Processing video: sgbc5n.mpg
[[19  5 20 39  7 18  5  5 14 39  2 25 39  3 39  6  9 22  5 39 14 15 23]]


In [6]:
import math

total_videos = count_what('set1','videos')

batch_size = 2

train_size = int(total_videos * 0.9)
test_size = total_videos - train_size

train_batch = math.ceil(train_size / batch_size)
test_batch = math.ceil(test_size / batch_size)

print('Train:',train_size)
print('Test:',test_size)

print("Steps per epoch:",train_batch)
print("Validation steps per epoch:", test_batch)

Train: 3597
Test: 400
Steps per epoch: 1799
Validation steps per epoch: 200


In [7]:
set1_data = tf.data.Dataset.list_files('./set1/Videos/*.mpg')
set1_data = set1_data.shuffle(500, reshuffle_each_iteration=False)
set1_data = set1_data.map(mappable_function)
set1_data = set1_data.padded_batch(batch_size, padded_shapes=([75, 46, 140, 1],[1,40]))
set1_data = set1_data.prefetch(tf.data.AUTOTUNE)
set1_data = set1_data.repeat()
# Added for split 
set1_train = set1_data.take(train_batch)
set1_test = set1_data.skip(train_batch)

In [8]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

batch_size  = 1
height, width, channels = 46, 140, 1

# Input layer
input_layer = Input(batch_shape=(batch_size, None, height, width, channels))

# Convolutional layers
x = Conv3D(128, 3, padding='same')(input_layer)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(256, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

x = Conv3D(75, 3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPool3D((1, 2, 2))(x)

# Flatten and TimeDistributed layer
x = TimeDistributed(Flatten())(x)

# Attention layer
# attention_output = Attention(use_scale=True, score_mode='dot')([x, x])

# Bidirectional LSTMs
lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal', stateful = True, return_sequences=True))(x)
lstm_output = Dropout(0.5)(lstm_output)

lstm_output = Bidirectional(LSTM(128, kernel_initializer='Orthogonal', stateful = True, return_sequences=True))(lstm_output)
lstm_output = Dropout(0.5)(lstm_output)

# Attention layer
attention_output = Attention(use_scale=True, score_mode='dot')([lstm_output, lstm_output])

# Dense output layer
output_layer = Dense(char_to_num.vocabulary_size() + 1, kernel_initializer='he_normal', activation='softmax')(attention_output)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)

# Print the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (1, None, 46, 140, 1)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (1, None, 46, 140,     │          3,584 │ input_layer[0][0]      │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (1, None, 46, 140,     │              0 │ conv3d[0][0]           │
│                           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (1, None, 23, 70, 128) │              0 │ activation[0][0]       │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (1, None, 23, 70, 256) │        884,992 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (1, None, 23, 70, 256) │              0 │ conv3d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_1           │ (1, None, 11, 35, 256) │              0 │ activation_1[0][0]     │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (1, None, 11, 35, 75)  │        518,475 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (1, None, 11, 35, 75)  │              0 │ conv3d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_2           │ (1, None, 5, 17, 75)   │              0 │ activation_2[0][0]     │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (1, None, 6375)        │              0 │ max_pooling3d_2[0][0]  │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (1, None, 256)         │      6,660,096 │ time_distributed[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (1, None, 256)         │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (1, None, 256)         │        394,240 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (1, None, 256)         │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)

 Total params: 8,471,925 (32.32 MB)

 Trainable params: 8,471,925 (32.32 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
optimizer = Adam(
    learning_rate=0.000179,  # Lower initial LR for stable CTC convergence
    beta_1=0.9,
    beta_2=0.999,
    clipnorm=1.0  # Prevent gradient explosion
)

def lr_scheduler(epoch, lr):
    if epoch < 59:  # Warmup phase
        return lr * 1.2
    elif (epoch+1) % 5 == 0:  # Step decay
        return lr * 0.5
    else:
        return lr

class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_loss = float('inf')
        self.wait = 0

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.wait = 0  # Reset counter
        else:
            self.wait += 1  # Increment counter

        if self.wait >= self.patience:
            print("\nEarly stopping triggered.")
            return True  # Stop training
        return False

import json

# Define checkpoint directory
DIR = WORKING_DIR + "/models/state"
os.makedirs(DIR, exist_ok=True)

def save_training_state(model, epoch):
    """
    Saves model weights, optimizer state, and training configs.
    """
    # Save model weights
    model.save_weights(os.path.join(DIR, "model_f25_set1.weights.h5"))
    
     # Save state
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    checkpoint.save(os.path.join(DIR, "checkpoint"))
    
    # Save epoch info
    with open(os.path.join(DIR, "epoch.txt"), "w") as f:
        f.write(str(epoch))

    print(f"Model wieghts saved at epoch {epoch+1}\n")



def load_training_state(model):
    """
    Loads model weights, optimizer state, and training configs.
    """
    try:
        # Load model weights
        model.load_weights(os.path.join(DIR, "model_f25_set1.weights.h5"))
        print("Loaded model weights.")
        
        checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
        
        checkpoint.restore(tf.train.latest_checkpoint(DIR))
        print("Loaded model and optimizer state.")

        # Load last saved epoch
        with open(os.path.join(DIR, "epoch.txt"), "r") as f:
            start_epoch = int(f.read()) + 1   # Resume from next epoch
        print(f"Resuming training from epoch {start_epoch + 1}")

    except Exception as e:
        print("No checkpoint found. Starting from scratch.")
        start_epoch = 0  # Start from the beginning

    return start_epoch

class ProduceExample: 
    def __init__(self, dataset):
        self.dataset = dataset.as_numpy_iterator()
    
    def produce(self, model):
        decode = []
        frames, alignments = self.dataset.next()
        batch_size =  frames.shape[0]
        for i in range(batch_size):
            frames_i = frames[i]  # Get video frames
            alignment = alignments[i]  # Get ground truth alignment
            # video_length = batch_video_lengths[i]  # Total frames in video
    
            # Split video into chunks
            chunks = split_video_into_chunks(frames_i)
    
            # Reset LSTM states before processing each video
            for layer in model.layers:
                if isinstance(layer, tf.keras.layers.Bidirectional):
                    layer.reset_states()
    
            all_logits = []
            input_lengths = []
    
            for chunk in chunks:
                chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
                logits = model(chunk, training=True)  # Forward pass
                all_logits.append(logits)
                input_lengths.append(logits.shape[1])  # Time steps in logits

            # Combine chunks into one continuous sequence
            all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)
            input_length = tf.convert_to_tensor([sum(input_lengths)], dtype=tf.int32)
            
            decoded = tf.keras.backend.ctc_decode(all_logits, input_length, greedy=False)[0][0].numpy()
            decode.append(decoded)
         
        for x in range(len(decode)):           
            print('Original:', tf.strings.reduce_join(num_to_char(alignments[x])).numpy().decode('utf-8'))
            print('Prediction:', tf.strings.reduce_join(num_to_char(decode[x])).numpy().decode('utf-8'))
            print('~'*100)


def CTCLoss_single_video(y_true, y_pred):
    
    #y_true = tf.squeeze(y_true, axis=0)  # Remove batch dimension (if any)
    # print(y_true.shape)
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int32")
    
    input_length = tf.shape(y_pred)[1]  # Total time steps in logits
    label_length = tf.shape(y_true)[1]  # Length of the ground truth sequence

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int32")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int32")

    # input_length = tf.expand_dims(input_length, axis=0)  # Make it a scalar tensor
    # label_length = tf.expand_dims(label_length, axis=0)  # Make it a scalar tensor

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


In [12]:
DIR = WORKING_DIR + "/models/state"

load_training_state(model)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 66


65

In [10]:
def split_video_into_chunks(frames, frames_per_chunk=25):
    """
    Splits the full video into fixed-size chunks.
    Returns a list of frame chunks.
    """
    total_frames = len(frames)
    num_chunks = total_frames // frames_per_chunk

    # Split into fixed-size chunks
    chunks = np.array_split(frames[:num_chunks * frames_per_chunk], num_chunks)

    return np.array(chunks)

In [11]:
os.remove('/kaggle/working/models/state/model_f25_set1.weights.h5')

In [15]:
DIR = WORKING_DIR + "/models/state"
with open(os.path.join(DIR, "epoch.txt"), "r") as f:
        epoch = f.readlines()
        print(epoch[0])

61


In [11]:
DIR = WORKING_DIR + "/models/state"
epoch = 64
with open(os.path.join(DIR, "epoch.txt"), "w") as f:
        f.write(str(epoch))

In [ ]:
# previously the model(trained in chunks of 25) was trained for 41 epochs(40 is printed because of 0 based indexing)
# now first we will train the set1 for 1 epoch 

In [11]:

def train_step(model, optimizer, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]  # Number of videos in the batch
    total_loss = 0.0  # Accumulate batch loss

    with tf.GradientTape() as tape:
        
        for i in range(batch_size):
            frames = batch_frames[i]  # Get video frames
            alignment = batch_alignments[i]  # Get ground truth alignment
            # video_length = batch_video_lengths[i]  # Total frames in video

            # Split video into chunks
            chunks = split_video_into_chunks(frames, frames_per_chunk)

            # Reset LSTM states before processing each video
            for layer in model.layers:
                if isinstance(layer, tf.keras.layers.Bidirectional):
                    layer.reset_states()

            all_logits = []
            input_lengths = []

            for chunk in chunks:
                chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
                logits = model(chunk, training=True)  # Forward pass
                all_logits.append(logits)
                input_lengths.append(logits.shape[1])  # Time steps in logits

            # Combine chunks into one continuous sequence
            all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)
            input_length = tf.reduce_sum(input_lengths) * tf.ones(shape=(1, 1), dtype="int32")

            # Compute loss for this video
            loss = CTCLoss_single_video(alignment, all_logits)
            total_loss += loss  # Accumulate loss

        # Compute batch loss
        batch_loss = total_loss / batch_size  # Average loss across batch

    # Compute gradients and update weights
    grads = tape.gradient(batch_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Update model weights

    return batch_loss 

def train_model(model, train_dataset, test_dataset, num_batches, optimizer, epochs):

    example = ProduceExample(test_dataset)
    early_stopping = EarlyStopping()
    start_epoch = load_training_state(model)
    
    for epoch in range(start_epoch, epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        lr = 0.000179
        optimizer.learning_rate.assign(lr)
        # lr = optimizer.learning_rate.numpy()
        print(f"Current Learning Rate: {lr:.6f}")

        total_epoch_loss = 0.0
        # Number of batches per epoch
        for batch_idx in tqdm(range(num_batches)):
            batch_frames, batch_alignments = train_dataset.as_numpy_iterator().next()
            # data = dataset.next()
            # batch_frames = data[0]
            # batch_alignments = data[1]
            loss = train_step(model, optimizer, batch_frames, batch_alignments)
            total_epoch_loss += loss.numpy().item()
            
        avg_epoch_loss = total_epoch_loss / num_batches  # Compute average loss
        print(f"Epoch {epoch+1} Completed. Average Loss: {avg_epoch_loss:.4f}\n")

        if early_stopping.step(avg_epoch_loss):
            break

        if early_stopping.wait == 0:
            save_training_state(model, epoch)

        example.produce(model)


In [ ]:
srbb3p.mpg
sgbc5n.mpg
bgba9a.mpg

In [20]:
train_model(model, set1_train, train_batch, optimizer, epochs = 42)

Loaded model weights.
Resuming training from epoch 42

Epoch 42/42
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Processing video: brwazs.mpg
Processing video: lbad8a.mpg
Processing video: pgwk8p.mpg
Processing video: prio8s.mpg
Processing video: pbac1a.mpg
Processing video: sbwh4p.mpg
Processing video: bgwh9a.mpg
Processing video: lwir1p.mpg
Processing video: lwwl8s.mpg
Processing video: lgwg3p.mpg
Processing video: lbwq9s.mpg
Processing video: lrwl3n.mpg
Processing video: braf8a.mpg
Processing video: sbbuza.mpg
Processing video: prbd1p.mpg
Processing video: lrwr7n.mpg
Processing video: sbat6s.mpg
Processing video: sbbg9n.mpg
Processing video: pbwc5n.mpg
Processing video: bwif7s.mpg
Processing video: bwis5s.mpg
Processing video: pgad7n.mpg
Processing video: pgax8n.mpg
Processing video: swbv1s.mpg
Processing video: lbax6n.mpg
Processing video: srwu8s.mpg
Processing video: priczp.mpg
Processing video: braf6p.mpg
Processing video: srwi1a.mpg
Processing video: brwm6n.mpg
Processing video: srbo2s.mpg
Processing video: sria6s.mpg
Processing video: sgai3s.mpg
Processing video: bgit3s.mpg
Processing vid

In [27]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 46)

Loaded model weights.
Resuming training from epoch 43

Epoch 43/46
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 43 Completed. Average Loss: 52.8429

Model wieghts saved at epoch 43

Original: bin white by g six now
Prediction: bin ren an h eir noasn
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 44/46
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 44 Completed. Average Loss: 45.9544

Model wieghts saved at epoch 44

Original: set green by c eight again
Prediction: set hute it c togr aogawn
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 45/46
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 45 Completed. Average Loss: 41.1426

Model wieghts saved at epoch 45

Original: bin red by m five again
Prediction: bea re at e fine soi
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 46/46
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 46 Completed. Average Loss: 37.7706

Model wieghts saved at epoch 46

Original: lay red in x seven now
Prediction: lay gren at f fhure agawn
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [38]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 47)

Loaded model weights.
Resuming training from epoch 47

Epoch 47/47
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 47 Completed. Average Loss: 34.0512

Model wieghts saved at epoch 47

Original: set green by p one soon
Prediction: sen gren at s tioe sgain
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin red at m one again
Prediction: bin red ith h five noawn
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [15]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 55)

Loaded model weights.
Resuming training from epoch 48

Epoch 48/55
Current Learning Rate: 0.000120


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 48 Completed. Average Loss: 30.3284

Model wieghts saved at epoch 48

Original: set red at h three now
Prediction: set re with a toe now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: set green in b four now
Prediction: set gren it b siv aow
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 49/55
Current Learning Rate: 0.000144


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 49 Completed. Average Loss: 26.7107

Model wieghts saved at epoch 49

Original: set blue at t three soon
Prediction: set blue ith s tire aoaie
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin red at s four soon
Prediction: bin blue at s zero soan
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 50/55
Current Learning Rate: 0.000173


...

Epoch 53 Completed. Average Loss: 17.1001

Model wieghts saved at epoch 53

Original: bin green by h four soon
Prediction: bin gren at t one son
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay red at y one now
Prediction: lay red iy y one now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 54/55
Current Learning Rate: 0.000358


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 54 Completed. Average Loss: 16.1024

Model wieghts saved at epoch 54

Original: set white at b four please
Prediction: set white it a fhre please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin blue by s two please
Prediction: bin blue bt m two please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 55/55
Current Learning Rate: 0.000358


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 55 Completed. Average Loss: 14.3893

Model wieghts saved at epoch 55

Original: set red at n five soon
Prediction: set red with u five son
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin red at s seven again
Prediction: bin red by s tere again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [23]:
# train_dataset = set1_train.as_numpy_iterator()
# test_dataset = set1_test.as_numpy_iterator()

In [ ]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 65)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 56

Epoch 56/65
Current Learning Rate: 0.000430


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 56 Completed. Average Loss: 13.7638

Model wieghts saved at epoch 56

Original: place white with d seven now
Prediction: place white with o seven now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin blue with s nine please
Prediction: bin blue with l nine please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 57/65
Current Learning Rate: 0.000516


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 57 Completed. Average Loss: 13.5251

Model wieghts saved at epoch 57

Original: place red by x one again
Prediction: place red by d oe agaon
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin red in s two again
Prediction: bin red in r two again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 58/65
Current Learning Rate: 0.000619


...

  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 59 Completed. Average Loss: 15.2859

Original: lay blue at q three again
Prediction: lay blue at u zere again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin white by t five please
Prediction: bin white by l fine please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 60/65
Current Learning Rate: 0.000372


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 60 Completed. Average Loss: 10.8222

Model wieghts saved at epoch 60

Original: place red at i nine please
Prediction: place red at i nine please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay red by k nine again
Prediction: lay wrete at k nine again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 61/65
Current Learning Rate: 0.000372


  0%|          | 0/1799 [00:00<?, ?it/s]

In [17]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 65)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 63

Epoch 63/65
Current Learning Rate: 0.000358


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 63 Completed. Average Loss: 10.2056

Model wieghts saved at epoch 63

Original: bin red in z six again
Prediction: bin red in n six again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: set blue by h zero now
Prediction: set blue by b zero now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 64/65
Current Learning Rate: 0.000358


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 64 Completed. Average Loss: 9.3983

Model wieghts saved at epoch 64

Original: lay blue at x eight soon
Prediction: lay blue at g eight son
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: place blue in o one now
Prediction: place blue in o one pow
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 65/65
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 65 Completed. Average Loss: 6.9541

Model wieghts saved at epoch 65

Original: bin green at g nine now
Prediction: bin gren at g thne now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: place red by p seven now
Prediction: place red by v zeron now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 70)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 66

Epoch 66/70
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 66 Completed. Average Loss: 5.2758

Model wieghts saved at epoch 66

Original: bin blue by l nine please
Prediction: bin blue by l nine please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: place white in v seven now
Prediction: place white in b seven now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 67/70
Current Learning Rate: 0.000100


  0%|          | 0/1799 [00:00<?, ?it/s]

In [ ]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 70)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 67

Epoch 67/70
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 67 Completed. Average Loss: 4.9404

Model wieghts saved at epoch 67

Original: lay white by l six sp please
Prediction: lay white by s six sp please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay red by y six soon
Prediction: lay red by y six son
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 68/70
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 68 Completed. Average Loss: 4.4662

Model wieghts saved at epoch 68

Original: set green in b nine again
Prediction: set gren in b nine again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay blue with k seven again
Prediction: lay blue with k seven again
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 69/70
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 69 Completed. Average Loss: 3.9752

Model wieghts saved at epoch 69

Original: bin red by m five please
Prediction: bin red by l five please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: bin white sp by g seven soon
Prediction: bin white sp bp g seven soaoin
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 70/70
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

In [12]:
train_model(model, set1_train, set1_test, train_batch, optimizer, epochs = 71)

Loaded model weights.
Loaded model and optimizer state.
Resuming training from epoch 71

Epoch 71/71
Current Learning Rate: 0.000179


  0%|          | 0/1799 [00:00<?, ?it/s]

Epoch 71 Completed. Average Loss: 3.4036

Model wieghts saved at epoch 71

Original: set white at h six now
Prediction: set white at h six now
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: set white with i six please
Prediction: set white with i six please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [13]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [13]:
from jiwer import wer,mer

def test_step(model, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]
    # input_length = batch_frames.shape[1]
    total_loss = 0.0
    # wer_i, ser_i = 0.0, 0.0

    for i in range(batch_size):
        frames_i = batch_frames[i]
        alignment = batch_alignments[i]
        # og = tf.strings.reduce_join([num_to_char(word) for word in alignment[0]]).numpy().decode('utf-8')

        chunks = split_video_into_chunks(frames_i, frames_per_chunk)

        # Reset LSTM states before processing each video
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Bidirectional):
                layer.reset_states()

        all_logits = []
        # input_lengths = []
        
        for chunk in chunks:
            chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
            # print(chunk.shape)
            logits = model.predict(chunk, verbose = 0)  # Training is False since it is used for testing and not for forward pass or training
            all_logits.append(logits)
            # input_lengths.append(logits.shape[1])  # Time steps in logits

        # Combine chunks into one continuous sequence
        all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)

        # decoded = tf.keras.backend.ctc_decode(all_logits, input_length=[input_length], greedy=True)[0][0].numpy()
        # predicted = tf.strings.reduce_join([num_to_char(word) for word in decoded[0]]).numpy().decode('utf-8')

        # wer_score = wer(og, predicted)
        # ser_score = mer(og, predicted)

        # wer_i += wer_score
        # ser_i += ser_score

        # print("Allignment before passing to ctcloss: ", alignment)
        # Compute loss for this video
        loss = CTCLoss_single_video(alignment, all_logits)
        total_loss += loss  # Accumulate loss

    batch_loss = total_loss / batch_size
    # batch_wer = wer_i / batch_size
    # batch_ser = ser_i / batch_size
    # print("done")

    return batch_loss
        

In [14]:
from jiwer import wer,mer

def test_step(model, batch_frames, batch_alignments, frames_per_chunk=25):

    batch_size = batch_frames.shape[0]
    input_length = batch_frames.shape[1]
    total_loss = 0.0
    wer_i, ser_i = 0.0, 0.0

    for i in range(batch_size):
        frames_i = batch_frames[i]
        alignment = batch_alignments[i]
        og = tf.strings.reduce_join([num_to_char(word) for word in alignment[0]]).numpy().decode('utf-8')

        chunks = split_video_into_chunks(frames_i, frames_per_chunk)

        # Reset LSTM states before processing each video
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Bidirectional):
                layer.reset_states()

        all_logits = []
        # input_lengths = []
        
        for chunk in chunks:
            chunk = tf.expand_dims(chunk, axis=0)  # Add batch dimension
            # print(chunk.shape)
            logits = model.predict(chunk, verbose = 0)  # Training is False since it is used for testing and not for forward pass or training
            all_logits.append(logits)
            # input_lengths.append(logits.shape[1])  # Time steps in logits

        # Combine chunks into one continuous sequence
        all_logits = tf.concat(all_logits, axis=1)  # Shape: (1, total_frames, num_classes)

        decoded = tf.keras.backend.ctc_decode(all_logits, input_length=[input_length], greedy=True)[0][0].numpy()
        predicted = tf.strings.reduce_join([num_to_char(word) for word in decoded[0]]).numpy().decode('utf-8')

        wer_score = wer(og, predicted)
        ser_score = mer(og, predicted)

        wer_i += wer_score
        ser_i += ser_score

        # print("Allignment before passing to ctcloss: ", alignment)
        # Compute loss for this video
        loss = CTCLoss_single_video(alignment, all_logits)
        total_loss += loss  # Accumulate loss

    batch_loss = total_loss / batch_size
    batch_wer = wer_i / batch_size
    batch_ser = ser_i / batch_size
    # print("done")

    return batch_loss, batch_wer, batch_ser
        

In [19]:


DIR = WORKING_DIR + "/models/state"

def test_model(model, test_dataset, num_batches):

    # dataset = test_dataset.as_numpy_iterator()
    
    model.load_weights(os.path.join(DIR, "model_f25_set1.weights.h5"))
    print("Loaded model weights.")

    total_loss = 0.0
    wer_score, ser_score = 0.0, 0.0

    for batch_idx in tqdm(range(num_batches)):
        d = test_dataset.as_numpy_iterator().next()
        frames = d[0]
        alignments = d[1]
        # print(alignments.shape)
        loss, batch_wer, batch_ser = test_step(model, frames, alignments)
        total_loss += loss
        wer_score += batch_wer
        ser_score += batch_ser

    avg_loss = total_loss / num_batches
    avg_wer = wer_score / num_batches
    avg_ser = ser_score / num_batches

    print(f"Testing Completed. Average Loss: {float(avg_loss):.4f}\n")
    print(f"Word Error Rate: {avg_wer:.4f}\n")
    print(f"Sentence Error Rate: {avg_ser:.4f}\n")

In [18]:

test_model(model, set1_test, test_batch)

Loaded model weights.


  0%|          | 0/200 [00:00<?, ?it/s]

Testing Completed. Average Loss: 4.9611

Word Error Rate: 0.0973

Sentence Error Rate: 0.0972



In [16]:

test_model(model, set1_train, train_batch)

Loaded model weights.


  0%|          | 0/1799 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
framesi, alignmentsi = set1_train.as_numpy_iterator().next()

In [17]:
test_model(model, set1_test, test_batch)

Loaded model weights.


  0%|          | 0/200 [00:00<?, ?it/s]

Testing Completed. Average Loss: 2.1802

Word Error Rate: 0.0233

Sentence Error Rate: 0.0232



In [20]:
test_model(model, set1_train, train_batch)

Loaded model weights.


  0%|          | 0/1799 [00:00<?, ?it/s]

Testing Completed. Average Loss: 2.2406

Word Error Rate: 0.0281

Sentence Error Rate: 0.0281

